<a href="https://colab.research.google.com/github/y-patankar/CPSC597/blob/main/CreditCard_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mlg-ulb/creditcardfraud
!unzip creditcardfraud.zip

 64% 42.0M/66.0M [00:00<00:00, 90.9MB/s]
100% 66.0M/66.0M [00:00<00:00, 111MB/s] 
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [5]:
import pandas as pd
df= pd.read_csv('creditcard.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [8]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# normalise the amount column
df['normAmount'] = StandardScaler().fit_transform(np.array(df['Amount']).reshape(-1, 1))

# drop Time and Amount columns as they are not relevant for prediction purpose
df = df.drop(['Time', 'Amount'], axis = 1)

# as you can see there are 492 fraud transactions.
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

# split into 70:30 ration
train, test, val = df[:240000], df[240000:262000], df[262000:]
train['Class'].value_counts(), test['Class'].value_counts(), val['Class'].value_counts()

# describes info about train and test set
# print("Number transactions X_train dataset: ", X_train.shape)
# print("Number transactions y_train dataset: ", y_train.shape)
# print("Number transactions X_test dataset: ", X_test.shape)
# print("Number transactions y_test dataset: ", y_test.shape)


(0    239563
 1       437
 Name: Class, dtype: int64,
 0    21963
 1       37
 Name: Class, dtype: int64,
 0    22789
 1       18
 Name: Class, dtype: int64)